In [3]:
import pandas as pd
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# Load data
df = pd.read_csv('/Users/rishithaperepally/Desktop/BBData.csv')

In [4]:
df["course description"] = ""

In [5]:
df

,New_ID,STUDID,Course Selected,Nationality,School,Demographic Data,Age,Modes of Learning,Session,Difficulty Level,Course Recommended,Course Rating,field of interest,cgpa,course description
0,0,1,Young App Developer,Bahrain,Coursetakers,Female,13,online,24,2,Robotics Innovative Learning,5,ELECTRONICS,2.9,
1,1,1,Young Entrepreneur,Bahrain,Coursetakers,Female,13,online,96,4,Robotics Innovative Learning,5,ELECTRONICS,2.9,
2,2,1,Young Honcho,Bahrain,Coursetakers,Female,13,offline,48,3,Python,1,ELECTRONICS,2.9,
3,3,1,Artificial Intelligence for Non Programmers,Bahrain,Coursetakers,Female,13,online,48,3,Technology Innovative Learning,5,ELECTRONICS,2.9,
4,4,2,Tech Innovator,USA,Diyafah International School Abu Dhabi,Male,15,offline,48,3,Robotics Innovative Learning,4,STEM,3.3,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10314,10314,3701,Scratch Coding,Qatar,NIMS,Female,10,hybrid,8,1,Graphics Designing,3,ROBOTICS,4.0,
10315,10315,3702,3D Designing & Modelling,Armenia,Dubai National School,Male,11,hybrid,24,2,App Development,3,ASTRONOMY,2.5,
10316,10316,3703,Tech Innovator,Egypt,NIMS,Male,10,offline,24,2,App Development,3,ARTIFICIAL INTELLIGENCE,3.8,
10317,10317,3704,French Language Course,Palestine,Saad ebn abi waqas,Female,12,offline,8,1,3D Designing,4,CODING,3.3,


In [17]:

course_dict = {
    "Introduction to Python": "This course provides an introduction to the Python programming language and its syntax.",
    "Data Structures and Algorithms": "This course covers data structures like arrays, stacks, queues, linked lists, trees, and graphs along with algorithms for searching, sorting, and traversal.",
    "Web Development": "This course covers the basics of web development including HTML, CSS, JavaScript, and back-end programming with Flask or Django.",
    "stem jr":"Introduction to Coding,Robotics,3D PrintingGame Development,Cybersecurity,Artificial Intelligence.",
    "stem Sr":"Introduction to Coding Robotics and Engineering Science Exploration Mathematics Basics Introduction to Electronics game development Data Science Advanced Robotics.",
    "web development": "Introduction to Web Design,Web Design and Development,Interactive Web Development,Full-Stack Web Development.",
    "3D Designing & Printing": "Introduction to 3D Designing and Printing,Intermediate 3D Designing and Printing,Advanced 3D Designing and Printing ,Engineering and Prototyping,Entrepreneurship and Marketing.",
    "App development": "Introduction to Coding,App Development Basics,Game Development,App Design,Advanced App Development,App Marketing and Monetization,Group Project.",
    "augmented and virtual reality": "Introduction to Augmented Reality,Virtual Reality Game Design,3D Modeling and Animation,AR and VR Storytelling,AR and VR for Education,Augmented Reality Art,Virtual Reality Cinematography,Augmented Reality Gaming,Virtual Reality Hardware,AR and VR Ethics.",
    "Html Programming": "Introduction to HTML,HTML Forms,CSS Basics,Responsive Web Design,HTML5 and CSS3,Advanced HTML and CSS,JavaScript and HTML,JavaScript and CSS,Introduction to Web Development.",
    "Data Analysis with python": "Introduction to Python,Data Types and Variables,Basic Data Analysis with Python,Data Visualization with Python,Data Cleaning and Preparation,Statistical Analysis with Python,Machine Learning with Python,Big Data Analysis with Python.",
    "Public Speaking & Entrepreneurship": "Introduction to Public Speaking,Entrepreneurship 101,Effective Communication,Business Basics,Storytelling,Pitching,Marketing and Advertising,Social Entrepreneurship,Public Speaking for Success,Financial Literacy.",
    "Electronic Program": "Introduction to Coding,Introduction to Robotics,Web Design,Game Development,Mobile App Development,Cybersecurity,3D Printing,Digital Art and Design,Data Science,Video Production.",
    "C++ Programming": "Introduction to Programming with Scratch,Beginner C++ Programming,Intermediate C++ Programming,Advanced C++ Programming.",
    "Virtual Robotics and mechatronics": "Introduction to Robotics,Lego Robotics,Advanced Robotics,Mechatronics,Robotics Competition Preparation.",
    "Drone Technology and Aerodynamics": "Introduction to Drones,Aerodynamics Basics,Drone Design,Drone Piloting,Drone Racing,Advanced Drone Design,Ethics and Safety.",
    "Virtual Robotics ": "Introduction to Robotics,Robot Design and Construction,Advanced Robotics Programming,Autonomous Robotics,Robot Challenges and Competitions.",
    "Game Development - Block Based": "Introduction to Game Development, Intermediate Game Development ,Advanced Game Development,Game Design and Development, Mobile Game Development,Virtual Reality Game Development.",
    "Game and App Development": "Introduction to Scratch ,Intermediate Scratch,Introduction to App Development,Intermediate App Development ,Introduction to Game Development,Intermediate Game Development,Game Design and Storytelling.",
    "App Design and Development ": "Introduction to App Design and Development,App Development Fundamentals,Advanced App Development,Mobile App Development,App Design Workshop,Game Development,Virtual Reality Development.",
    "App Development ": "Scratch Programming,App Inventor,Swift Playgrounds,Code.org,Tynker,Codeacademy.",
    "Design Thinking and Entrepreneur Program ": "Introduction to Design Thinking,Entrepreneurship 101,Product Design for Kids,Business Planning for Kids,Coding and Entrepreneurship,Maker and Innovation Lab.",
    "Artificial Intelligence for Non Programmers": "Introduction to AI,Natural Language Processing,Computer Vision,Ethics in AI,AI and Gaming,Robotics,Data Science,Machine Learning,AI and Creativity,AI and Healthcare.",
    "Artificial Intelligence And Machine Learning": "Introduction to Coding,Introduction to Robotics,Artificial Intelligence for Kids ,Introduction to Machine Learning,Data Science for Teens,Advanced Machine Learning and Deep Learning .",
    "Tech Innovator": "emerging technologies, Artificial Intelligence, Coding, 3D Designing and App Development.",
    "Young Honcho": "Artificial Intelligence, Machine Learning, Data Science, Design Thinking and Entrepreneurship.",

       "Technology Onsite": "Robotics, Coding, Internet of Things (IoT), Drone Technology, 3D Designing & Printing, Space STEM, Augmented Reality,.",
        "Young Entrepreneur": "Coding, Applications of coding, Public Speaking, Oratory Skills, Extempore, Design Thinking, Entrepreneurship.",

        "Young Innovator": "Robotics, Coding, App Development, 3D Designing and Basic Electronics.",

        "Young Product Designer": "product designing, experience designing, information architect, experience architect.",
        "Young Designer": " STEM,Graphic Designing, Animation, Design Thinking and Entrepreneurship,Virtual Internship.",
        "Young Architect": "STEM,3D tools, graphic designing, product designing and even game designing,virtual/onsite internship.",
        "Young Game Developer": "developing and designing games,STEM.",
    
        "Young App Developer": "Mobile App developer,problem-solving and analytics skills, prototype real-time app-based solutions to problems identified through design thinking.",
   
        "Young Web Developer": "STEM,HTML, CSS, JavaScript",
    
        "Young Coder": "coding like conditions, loops, functions,games, apps, and AI models to teach machines.",
   
        "3D Designing & Modelling": "design complex dynamic and static models using creativity and 3D designing tools",
    
        "Graphic Designing": "concepts like layout techniques, typography, color theory to design flyers, magazines, logos, videos and more.",
    
        "Robotics Innovative Learning" : " design & build complex products, machines, systems, or structures and prototypes",
    
        "LEGO Robotics": "structure design and control to advanced concepts like sensory feedback control. In this course you will learn to develop robots to solve various problems by building and coding different robotic models.",
   
        "Arduino": "program robots using open source platforms, develop prototypes to solve challenging problems, integrate various different sensors and actuators to the microcontroller board, develop scripts in c and c++ to code and control the robot.",
    
        "VEX Robotics ": "programming, sensors, and automation, and they hone critical and computational thinking skills",
    
        "Python Coding": " fields like software development, artificial intelligence, data science, ",
    
        "JAVA": "fields like software development, application development, web based applications,",
        "Scratch Coding": "concepts of conditions, loops and functions. Learn the fundamentals of coding using a block based coding interface, develop animation, comics, games",
        "Scratch Junior" : "can program their own interactive stories and games",
        "Game Development with Unity": " 2D and 3D games of different genres like adventure, action, puzzle and much more.",
   
        "Microsoft Arcade": "using block-based and javascript coding. Understand and apply the concepts of game design process, level design, character control and much more.",
   
        "Game Dev with SCRATCH": " designing and developing games in block based coding interface, coding like conditions, loops, Functions and much more.",
    
        "Drone Technology": "design and working of drones, develop strategic plans to code and control drones to operates autonomously.",
    
    
        "Internet of Things": "start building IoT applications using a variety of sensors and microcontrollers modules, fundamentals in electronics.",
    
        "Artificial Intelligence": " getting familiar with the terminologies of AI and identifying different problems which can be solved using AI.",
    
        "Data Science": " scientific method of processing and analyzing data, generate insights to make data-driven decisions.",
    
        "Artificial Intelligence of Things (AIoT)": " composition of AI, IoT, Design Thinking, and Entrepreneurship.",
    
        "Metaverse": "learn about the fundamentals of the metaverse, design their 3D interactive space using virtual reality, augmented reality, and mixed reality, and cryptocurrency transactions in the world of the metaverse.",
    
        "Space STEM": " learn and understand the solar system, phases of moon and concepts of different forces, space mathematics. This course will help you explore the concepts in the field of astronomy, astrophysics.",
    }


In [20]:
#Loop over the rows in the dataset and fill in the course descriptions based on the course selected
for i, row in df.iterrows():
    course_selected = row["Course Selected"]
    if course_selected in course_dict:
        df.at[i, "course description"] = course_dict[course_selected]
    else:
        df.at[i, "course description"] = "No description available"


In [21]:
# Print the updated dataset with the new column
print(df)

       New_ID  STUDID                              Course Selected  \
0           0       1                          Young App Developer   
1           1       1                           Young Entrepreneur   
2           2       1                                 Young Honcho   
3           3       1  Artificial Intelligence for Non Programmers   
4           4       2                               Tech Innovator   
...       ...     ...                                          ...   
10314   10314    3701                               Scratch Coding   
10315   10315    3702                     3D Designing & Modelling   
10316   10316    3703                               Tech Innovator   
10317   10317    3704                       French Language Course   
10318   10318    3705                         Young Data Scientist   

      Nationality                                             School  \
0         Bahrain                                       Coursetakers   
1         Bahra

In [22]:
df

,New_ID,STUDID,Course Selected,Nationality,School,Demographic Data,Age,Modes of Learning,Session,Difficulty Level,Course Recommended,Course Rating,field of interest,cgpa,course description
0,0,1,Young App Developer,Bahrain,Coursetakers,Female,13,online,24,2,Robotics Innovative Learning,5,ELECTRONICS,2.9,"Mobile App developer,problem-solving and analy..."
1,1,1,Young Entrepreneur,Bahrain,Coursetakers,Female,13,online,96,4,Robotics Innovative Learning,5,ELECTRONICS,2.9,"Coding, Applications of coding, Public Speakin..."
2,2,1,Young Honcho,Bahrain,Coursetakers,Female,13,offline,48,3,Python,1,ELECTRONICS,2.9,"Artificial Intelligence, Machine Learning, Dat..."
3,3,1,Artificial Intelligence for Non Programmers,Bahrain,Coursetakers,Female,13,online,48,3,Technology Innovative Learning,5,ELECTRONICS,2.9,"Introduction to AI,Natural Language Processing..."
4,4,2,Tech Innovator,USA,Diyafah International School Abu Dhabi,Male,15,offline,48,3,Robotics Innovative Learning,4,STEM,3.3,"emerging technologies, Artificial Intelligence..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10314,10314,3701,Scratch Coding,Qatar,NIMS,Female,10,hybrid,8,1,Graphics Designing,3,ROBOTICS,4.0,"concepts of conditions, loops and functions. L..."
10315,10315,3702,3D Designing & Modelling,Armenia,Dubai National School,Male,11,hybrid,24,2,App Development,3,ASTRONOMY,2.5,design complex dynamic and static models using...
10316,10316,3703,Tech Innovator,Egypt,NIMS,Male,10,offline,24,2,App Development,3,ARTIFICIAL INTELLIGENCE,3.8,"emerging technologies, Artificial Intelligence..."
10317,10317,3704,French Language Course,Palestine,Saad ebn abi waqas,Female,12,offline,8,1,3D Designing,4,CODING,3.3,No description available


In [23]:
df.to_csv('/Users/rishithaperepally/Desktop/BBData1.csv', index=False)

In [28]:
df = df[['Course Selected','Difficulty Level','course description','field of interest']]

In [29]:
df.head(5)

,Course Selected,Difficulty Level,course description,field of interest
0,Young App Developer,2,"Mobile App developer,problem-solving and analy...",ELECTRONICS
1,Young Entrepreneur,4,"Coding, Applications of coding, Public Speakin...",ELECTRONICS
2,Young Honcho,3,"Artificial Intelligence, Machine Learning, Dat...",ELECTRONICS
3,Artificial Intelligence for Non Programmers,3,"Introduction to AI,Natural Language Processing...",ELECTRONICS
4,Tech Innovator,3,"emerging technologies, Artificial Intelligence...",STEM


In [31]:
# Removing spaces between the words (Lambda funtions can be used as well)

df['Course Selected'] = df['Course Selected'].str.replace(' ',',')
df['Course Selected'] = df['Course Selected'].str.replace(',,',',')
df['Course Selected'] = df['Course Selected'].str.replace(':','')
df['course description'] = df['course description'].str.replace(' ',',')
df['course description'] = df['course description'].str.replace(',,',',')
df['course description'] = df['course description'].str.replace('_','')
df['course description'] = df['course description'].str.replace(':','')
df['course description'] = df['course description'].str.replace('(','')
df['course description'] = df['course description'].str.replace(')','')

#removing paranthesis from skills columns 
df['field of interest'] = df['field of interest'].str.replace('(','')
df['field of interest'] = df['field of interest'].str.replace(')','')

<ipython-input-31-239dec70dd0d>:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['course description'] = df['course description'].str.replace('(','')
<ipython-input-31-239dec70dd0d>:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['course description'] = df['course description'].str.replace(')','')
<ipython-input-31-239dec70dd0d>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['field of interest'] = df['field of interest'].str.replace('(','')
<ipython-input-31-239dec70dd0d>:15: FutureWarning: The default value 

In [32]:
df.head()

,Course Selected,Difficulty Level,course description,field of interest
0,"Young,App,Developer",2,"Mobile,App,developer,problem-solving,and,analy...",ELECTRONICS
1,"Young,Entrepreneur",4,"Coding,Applications,of,coding,Public,Speaking,...",ELECTRONICS
2,"Young,Honcho",3,"Artificial,Intelligence,Machine,Learning,Data,...",ELECTRONICS
3,"Artificial,Intelligence,for,Non,Programmers",3,"Introduction,to,AI,Natural,Language,Processing...",ELECTRONICS
4,"Tech,Innovator",3,"emerging,technologies,Artificial,Intelligence,...",STEM


In [33]:
##tags column

In [35]:
df['tags'] = df['Course Selected']  + df['course description'] + df['field of interest']

In [36]:
df['tags'].iloc[1]

'Young,EntrepreneurCoding,Applications,of,coding,Public,Speaking,Oratory,Skills,Extempore,Design,Thinking,Entrepreneurship.ELECTRONICS'

In [37]:
new_df = df[['Course Selected','tags']]

In [38]:
new_df.head(5)

,Course Selected,tags
0,"Young,App,Developer","Young,App,DeveloperMobile,App,developer,proble..."
1,"Young,Entrepreneur","Young,EntrepreneurCoding,Applications,of,codin..."
2,"Young,Honcho","Young,HonchoArtificial,Intelligence,Machine,Le..."
3,"Artificial,Intelligence,for,Non,Programmers","Artificial,Intelligence,for,Non,ProgrammersInt..."
4,"Tech,Innovator","Tech,Innovatoremerging,technologies,Artificial..."


In [39]:
new_df['tags'] = df['tags'].str.replace(',',' ')

<ipython-input-39-7b50b13b07b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = df['tags'].str.replace(',',' ')


In [42]:
new_df['Course Selected'] = df['Course Selected'].str.replace(',',' ')

<ipython-input-42-2d6dcab7e8c9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Course Selected'] = df['Course Selected'].str.replace(',',' ')


In [69]:
new_df.rename(columns = {'course selected':'course_selected'}, inplace = True)

/Users/rishithaperepally/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [70]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower()) #lower casing the tags column

<ipython-input-70-4cf64f7b4680>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower()) #lower casing the tags column


In [71]:
new_df.head(5)

,course_name,tags,Course Seelected
0,Young App Developer,young app developermobil app develop problem-s...,Young App Developer
1,Young Entrepreneur,young entrepreneurcod applic of code public sp...,Young Entrepreneur
2,Young Honcho,young honchoartifici intellig machin learn dat...,Young Honcho
3,Artificial Intelligence for Non Programmers,artifici intellig for non programmersintroduct...,Artificial Intelligence for Non Programmers
4,Tech Innovator,tech innovatoremerg technolog artifici intelli...,Tech Innovator


In [72]:
new_df.shape

(10319, 3)

In [73]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()

In [74]:
##stemming process

In [75]:
import nltk #for stemming process
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
#defining the stemming function
def stem(text):
    y=[]
    
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

In [76]:
new_df['tags'] = new_df['tags'].apply(stem) #applying stemming on the tags column

<ipython-input-76-edae1a2ddc5e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem) #applying stemming on the tags column


In [77]:
##similarity measure

In [78]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [80]:
def recommend(Course_selected):
    course_index = new_df[new_df['course_selected'] == course].index[0]
    distances = similarity[course_index]
    course_list = sorted(list(enumerate(distances)),reverse=True, key=lambda x:x[1])[1:7]
    
    for i in course_list:
        print(new_df.iloc[i[0]].course_name)

In [81]:
recommend('Young App Developer') 

KeyError: 'course_selected'